# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
csv_path = os.path.join("..", "city_data.csv")
print(csv_path)
city_df = pd.read_csv(csv_path, encoding="utf-8")
city_df

../city_data.csv


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F
0,bafang,5.1563,10.1788,291.84,55.0,4.0,0.69,CM,1644356345,Bafang,65.642
1,prince rupert,54.3161,-130.3201,278.20,93.0,100.0,18.52,CA,1644356345,Prince Rupert,41.090
2,bubaque,11.2833,-15.8333,298.82,72.0,2.0,5.53,GW,1644356346,Bubaque,78.206
3,bluff,-46.6000,168.3333,288.04,85.0,94.0,4.56,NZ,1644356346,Bluff,58.802
4,ketchikan,55.3422,-131.6461,279.41,87.0,100.0,6.17,US,1644356346,Ketchikan,43.268
...,...,...,...,...,...,...,...,...,...,...,...
553,maroon town,18.3446,-77.7954,295.15,85.0,79.0,0.77,JM,1644356527,Maroon Town,71.600
554,atambua,-9.1061,124.8925,296.47,92.0,100.0,0.68,ID,1644356527,Atambua,73.976
555,port lincoln,-34.7333,135.8667,291.81,75.0,100.0,4.32,AU,1644356528,Port Lincoln,65.588
556,srednekolymsk,67.4500,153.6833,230.95,99.0,71.0,2.61,RU,1644356528,Srednekolymsk,-43.960


In [13]:
#(K − 273.15) × 9/5 + 32
#air_quality["london_mg_per_cubic"] = air_quality["station_london"] * 1.882
#city_df['max temp F'] = (city_df['max temp'] - 273.15) * 9/5 + 32
#city_df.head()


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F
0,bluff,-46.6000,168.3333,279.15,78.0,6.0,2.14,NZ,1644173770,Bluff,42.800
1,san policarpo,12.1791,125.5072,299.80,78.0,36.0,6.87,PH,1644173771,San Policarpo,79.970
2,vaini,-21.2000,-175.2000,297.24,94.0,100.0,5.66,TO,1644173771,Vaini,75.362
3,butaritari,3.0707,172.7902,300.59,77.0,34.0,6.81,KI,1644173771,Butaritari,81.392
4,carnarvon,-24.8667,113.6333,297.19,69.0,100.0,9.77,AU,1644173772,Carnarvon,75.272


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
weather_df = city_df.loc[(city_df['cloudiness'] == 0) & (city_df['windspeed'] < 10) & (city_df['max temp F'] < 80) & (city_df['max temp'] > 70)]
weather_df.count()

city          72
lat           72
lng           72
max temp      72
humidity      72
cloudiness    72
windspeed     72
country       72
date          72
name          72
max temp F    72
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
location_df = city_df[["lat", "lng"]]
weights = city_df["humidity"]
weights

0      78.0
1      78.0
2      94.0
3      77.0
4      69.0
       ... 
566    75.0
567    18.0
568    93.0
569    88.0
570    58.0
Name: humidity, Length: 571, dtype: float64

In [20]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location_df, weights=weights, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = weather_df
hotel_df['hotel name'] = ""
hotel_df

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_65971/403779039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['hotel name'] = ""


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F,hotel name
5,gat,31.6100,34.7642,285.88,83.0,0.0,2.50,IL,1644173772,Kiryat Gat,54.914,
6,jega,12.2175,4.3792,299.36,15.0,0.0,4.59,NG,1644173772,Jega,79.178,
9,bredasdorp,-34.5322,20.0403,294.77,73.0,0.0,3.60,ZA,1644173773,Bredasdorp,70.916,
13,zaragoza,41.6561,-0.8773,284.27,50.0,0.0,3.58,ES,1644173774,Zaragoza,52.016,
33,saint george,37.1041,-113.5841,288.41,24.0,0.0,0.45,US,1644173779,Saint George,59.468,
...,...,...,...,...,...,...,...,...,...,...,...,...
507,jaisalmer,26.9147,70.9181,292.43,33.0,0.0,2.30,IN,1644173925,Jaisalmer,66.704,
523,havre-saint-pierre,50.2334,-63.5986,258.48,51.0,0.0,7.20,CA,1644173931,Havre-St-Pierre,5.594,
530,tessalit,20.1986,1.0114,294.65,10.0,0.0,3.02,ML,1644173933,Tessalit,70.700,
548,talcahuano,-36.7167,-73.1167,292.52,63.0,0.0,5.85,CL,1644173939,Talcahuano,66.866,


In [63]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_search = ""
target_radius = 5000
target_type = ["hotel"]
lat = ""
lng = ""

In [64]:
from pprint import pprint

In [67]:
# for index, row in hotel_df[0:2].iterrows():
#     lat = row['lat']
#     lng = row['lng']
#     params = {
#     "location": f"{lat},{lng}",
#     "radius": target_radius,
#     "type": target_type,
#     "key": g_key
#     }
#     url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     response = requests.get(url, params=params).json()
#     print(f"{lat}, {lng}")

31.61, 34.7642
12.2175, 4.3792


In [70]:
hotel_names = []
for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    params = {
    "location": f"{lat},{lng}",
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    response = requests.get(url, params=params).json()

    try:
        hotel_names.append(response['results'][0]['name'])
    except:
        hotel_names.append("")
hotel_df["hotel name"]=hotel_names
hotel_df

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_65971/973166775.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"]=hotel_names


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F,hotel name,Hotel Name
5,gat,31.6100,34.7642,285.88,83.0,0.0,2.50,IL,1644173772,Kiryat Gat,54.914,Kiryat Gat,Kiryat Gat
6,jega,12.2175,4.3792,299.36,15.0,0.0,4.59,NG,1644173772,Jega,79.178,Jega,Jega
9,bredasdorp,-34.5322,20.0403,294.77,73.0,0.0,3.60,ZA,1644173773,Bredasdorp,70.916,Bredasdorp,Bredasdorp
13,zaragoza,41.6561,-0.8773,284.27,50.0,0.0,3.58,ES,1644173774,Zaragoza,52.016,Zaragoza,Zaragoza
33,saint george,37.1041,-113.5841,288.41,24.0,0.0,0.45,US,1644173779,Saint George,59.468,St. George,St. George
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,jaisalmer,26.9147,70.9181,292.43,33.0,0.0,2.30,IN,1644173925,Jaisalmer,66.704,Jaisalmer,Jaisalmer
523,havre-saint-pierre,50.2334,-63.5986,258.48,51.0,0.0,7.20,CA,1644173931,Havre-St-Pierre,5.594,Havre-Saint-Pierre,Havre-Saint-Pierre
530,tessalit,20.1986,1.0114,294.65,10.0,0.0,3.02,ML,1644173933,Tessalit,70.700,Achemelmel,Achemelmel
548,talcahuano,-36.7167,-73.1167,292.52,63.0,0.0,5.85,CL,1644173939,Talcahuano,66.866,Talcahuano,Talcahuano


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [73]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))